<a href="https://colab.research.google.com/github/meteor79/TPS/blob/master/TextRank_doc_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install newspaper3k # url 크롤링 패키지
!pip install jpype1
!pip install konlpy # 한글 형태소 분석기
!pip install scikit-learn # TF-IDF를 위한 머신러닝 패키지
!pip install html2text

     |████████████████████████████████| 215kB 2.7MB/s 
     |████████████████████████████████| 194kB 8.7MB/s 
     |████████████████████████████████| 7.4MB 5.3MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3357 sha256=9c6e2920693a390074dda3c3eafd570ceec27f618dd3bf6c886082deb3e116c0
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for feedparser: filename=feedparser-5.2.1-cp36-none-any.whl size=44940 sha256=c251356da7a7dbc885bf0d33c5e868f5dade33908608c04643c57cc3f36e9f31
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=b797308532e553bac8ae0a644652179b227b3b905467338e27bfdc17f296e032
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da206

In [0]:
# -*- coding: utf-8 -*-

from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import requests
import html2text


def getHtmlText(url):
    try:
        headers = {'User-agent': 'Mozilla/5.0'}
        page = requests.get(url, headers=headers)        
        html_code = page.content        
        h = html2text.HTML2Text()                 
        h.ignore_links = True
        h.ignore_images = True
        h.single_line_break = True                     
        text = h.handle(html_code.decode("utf-8"))
        return text
    except Exception as e:
        print(e)
        return null

# 문장 추출
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.Okt = Okt()
           
    def url2sentences(self, url):
        htmltext = getHtmlText(url)
        sentences = self.kkma.sentences(htmltext)
        # for idx in range(0, len(sentences)):
        #     if len(sentences[idx]) <= 10:
        #         sentences[idx-1] += (' ' + sentences[idx])
        #         sentences[idx] = ''        
        return sentences
  
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)      
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences

    def get_nouns(self, sentences):
        '''KoNLPy로 형태소 분석하기''' # --- ( ※ 3) 
        nouns = []
        for sentence in sentences:
            word_s = self.Okt.pos(sentence, norm=True, stem=True)
            for n, h in word_s:
                if not (h in ['Noun']): continue
                if h == 'Punctuation' and h2 == 'Number': continue
                nouns.append(n)
        return nouns


# TF-IDF 모델 생성 및 그래프 생성
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}



#TextRank 알고리즘 적용
class Rank(object):
    def get_ranks(self, graph, d=0.85): 
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 
            link_sum = np.sum(A[:,id])
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) 
        return {idx: r[0] for idx, r in enumerate(ranks)}


#TextRank Class 구현
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)
  
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        
        print(self.nouns)
        # self.graph_matrix = GraphMatrix()
        # self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        # self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)

        # self.rank = Rank()
        # self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        # self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)

        # self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        # self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()

        for idx in index:
            summary.append(self.sentences[idx])
        return summary

    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

        keywords = []
        index=[]
        
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)

        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])

        return keywords


In [0]:
def run_text_rank(url, summarize_num):
    textrank = TextRank(url)
    rows = textrank.summarize(summarize_num)
    print (rows)
    # for row in :  # 몇 줄로 요약할꺼야
    #     print(row)
    #     print('\n')
    #     print('keywords :')
    #     print(textrank.keywords())

In [0]:
run_text_rank("http://www.diningcode.com/profile.php?rid=BpZCgY9DffCt", 3)


TypeError: ignored